# Connect 4: Exploring Algorithmic Approaches

**Project Context:** Constraint Programming for Connect 4

This notebook outlines and explains various algorithmic approaches that can be considered for developing an intelligent agent to play Connect 4. While the core project might involve Constraint Programming (CP) for aspects like state validation, win condition checking, or perhaps even move generation under certain constraints, this document focuses on common game-playing algorithms that could leverage or complement a CP model.

## 1. Introduction to Connect 4

Connect 4 is a classic two-player connection game in which players first choose a color and then take turns dropping colored discs from the top into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs.

## 2. Constraint Programming (CP) in Connect 4

While the primary decision-making logic for our agents might rely on search algorithms (Minimax, Negamax) or learning (Reinforcement Learning), Constraint Programming offers a powerful declarative paradigm to model and enforce the rules and properties of the Connect 4 game. Instead of explicitly coding *how* to check conditions step-by-step, CP allows us to define the conditions (constraints) that must hold true.

In the context, CP can be strategically employed for several key tasks:

1.  **Board State Representation & Rule Enforcement:**
    *   **Model:** The 6x7 board can be represented using CP variables. For example, a 2D array `board[row][col]` where each variable's domain is `{0, 1, 2}` (representing Empty, Player 1, Player 2).
    *   **Gravity Constraint:** A fundamental rule. We can define constraints stating that if `board[r][c]` is non-empty (belongs to Player 1 or 2), then `board[r-1][c]` must also be non-empty, unless `r` is the bottom row (r=0). This ensures pieces stack correctly.
    *   **Column Capacity:** A constraint can limit the number of non-empty cells in any given column `c` to be at most 6.
    *   **Benefit:** While simple checks are often used in practice for these, using CP provides a formal, declarative model of the game's physics, which can be useful for validation or more complex reasoning.

2.  **Win Condition Checking:**
    *   **Horizontal:** For each player `P` and each possible starting position `(r, c)`, a constraint like: `board[r][c] == P AND board[r][c+1] == P AND board[r][c+2] == P AND board[r][c+3] == P`.
    *   **Vertical:** Similarly: `board[r][c] == P AND board[r+1][c] == P AND board[r+2][c] == P AND board[r+3][c] == P`.
    *   **Diagonal (Both directions):** Analogous constraints for diagonal lines.
    *   **Integration:** This CP-based check is essential for:
        *   **Minimax/Negamax:** Determining if a node is a terminal state (win/loss) in the search tree.
        *   **Reinforcement Learning:** Determining when an episode ends (game over) and assigning the appropriate terminal reward (+1 for win, -1 for loss) in the environment's `step` function.
        *   **All Agents:** Knowing when to stop the game loop.

3.  **Valid Move Identification / Generation:**
    *   Define constraints for a legal move in column `c`: `c >= 0 AND c < 7` AND `board[5][c] == 0` (assuming row 5 is the top row). The actual row a piece lands in depends on the gravity constraint.
    *   **Integration:** Used by the Random agent to pick from legal moves, and by Minimax/Negamax/RL agents to know the available actions from a given state.

4.  **Heuristic Feature Definition (Supporting Minimax/Negamax):**
    *   CP can define what constitutes strategically important patterns, even if calculating them uses procedural code for speed.
    *   **Example:** Define a constraint set representing "a Player 1 three-in-a-row with open space on both ends". The heuristic function could then try to count how many solutions exist for this constraint set on the current board.
    *   **Threat Detection:** Model "a column `c` where Player 2 wins if they play there next". This involves checking if placing Player 2's piece in `c` satisfies a win condition.
    *   **Integration:** CP helps formalize the *features* that the procedural `evaluate_board` function will count or check. Running a CP solver *inside* the heuristic for every node might be too slow, but CP informs the design.

**Summary of CP's Role:** In this project setup, CP is unlikely to be the main AI decision engine itself. Instead, it acts as a powerful **rule engine and state analyzer**. Its primary roles are:
*   **Robustly checking for terminal states (win/loss/draw)**, which is fundamental for all other algorithms.
*   **Formally defining game rules and valid states/moves**, ensuring correctness.
*   Potentially **defining complex features** used by the heuristics in search algorithms like Minimax/Negamax.

## 3. Naive Strategies

These strategies represent simpler, often non-optimal approaches to playing Connect 4. They serve as essential baselines for evaluating more complex algorithms.

### 3.1 Random

The Random agent embodies the simplest possible strategy. In any given board state, it identifies all legally valid moves (columns that are not full). From this set of valid moves, it selects one column completely at random and drops its piece there. It has no concept of strategy, winning lines, threats, or opponent intentions. Its primary purpose in benchmarking is to provide a lower bound on performance – any reasonable AI should significantly outperform a purely random player.

### 3.2 Rule-Based Heuristic

A Rule-Based Heuristic agent uses a predefined set of simple, prioritized rules to select its move. Unlike search algorithms that look ahead, it typically only evaluates the immediate consequences of potential moves based on these rules. Examples of rules might include:

1.  **Win:** If a move results in an immediate win, take it.
2.  **Block Win:** If the opponent has an immediate winning move, block it.
3.  **Set up Win:** If a move creates a guaranteed win on the next turn (e.g., two simultaneous threats), take it.
4.  **Block Setup:** Prevent the opponent from setting up a guaranteed win.
5.  **Center Preference:** Prefer playing in center columns (often strategically advantageous).
6.  **Build Lines:** Prefer moves that extend lines of 2 or 3 discs.

The agent checks these rules in order of priority and executes the first one that applies. If no specific rule applies, it might default to a simpler strategy like playing in the first available column (like the 'Always First' agent seen in the benchmarks) or choosing randomly. These agents are typically fast but can be easily exploited by opponents who look further ahead.

## 4. Tree Search Algorithms

Tree search algorithms are fundamental to game AI. They explore possible future game states by constructing a game tree, where nodes represent board states and edges represent moves. By evaluating the outcomes deep within the tree, they aim to find the optimal move from the current state, assuming the opponent also plays optimally (or near-optimally).

### 4.1 Minimax

The **Minimax** algorithm is a cornerstone of game-playing AI, particularly suited for two-player, zero-sum games with perfect information, like **Connect 4**. It provides a systematic way for an AI player to determine the optimal move in any given board state.

#### Core Concept: Simulating Connect 4 Futures

Minimax operates by exploring a *game tree* representing possible future states of the Connect 4 board.
*   **Nodes:** Each node in this tree corresponds to a specific configuration of the Connect 4 grid. The root node is the current board state.
*   **Edges:** Each edge represents a legal move – dropping a colored piece into one of the seven columns (provided the column is not full).
The algorithm recursively explores branches of this tree, simulating turns down to a predetermined search depth or until a game-ending state is reached.
*   **Terminal States:** In Connect 4, these are states where one player has achieved four pieces in a row (horizontally, vertically, or diagonally) resulting in a win/loss, or the board is completely full, resulting in a draw. These terminal states are assigned scores: typically +1 for an AI win, -1 for an opponent win, and 0 for a draw.

<center>
    <figure>
        <img src="benchmark_results\connect-4-game-tree-diagram.png" width="400">
        <figcaption>Example of a minimax Game Tree</figcaption>
    </figure>
</center>

#### Maximizing (AI) vs. Minimizing (Opponent) Players

Minimax assumes optimal play from both sides:
*   **Max (Our AI):** Aims to *maximize* the final score. When it's the AI's turn to move (represented by a 'Max' level in the tree), it analyzes the scores resulting from dropping a piece in each valid column and chooses the column leading to the highest possible score.
*   **Min (The Opponent):** Aims to *minimize* the AI's score. When it's the opponent's turn ('Min' level), the algorithm assumes the opponent will choose the column leading to the lowest possible score for the AI.
Scores from deeper levels (or terminal states) are propagated back up the tree. Max nodes take the maximum value of their children, while Min nodes take the minimum.

Minimax uses a *heuristic evaluation function* at its maximum search depth. This function estimates the "goodness" of a non-terminal board state for the AI based on the number of potential winning lines. The AI ultimately chooses the initial column drop that leads to the branch with the highest backed-up score, assuming the opponent always plays perfectly to counter.

#### Optimization: Alpha-Beta Pruning for Connect 4 Efficiency

Exploring the full game tree for Connect 4, even a few moves deep, involves analyzing a massive number of board states. **Alpha-Beta Pruning** is an essential optimization that significantly speeds up this process without changing the outcome. It avoids evaluating parts of the game tree that won't influence the final decision. It maintains two key values during the search:
*   **Alpha (α):** The best (highest) score the AI (Max) can currently guarantee itself on the path from the root to the current node.
*   **Beta (β):** The best (lowest) score the opponent (Min) can currently guarantee forcing on the path from the root to the current node.

Pruning happens in two scenarios relevant to Connect 4 evaluations:
1.  While exploring the opponent's potential moves (Min node): If the evaluation of a potential opponent move yields a score less than or equal to `alpha`, the AI knows the opponent could force a worse outcome for the AI than one already found (`alpha`). Therefore, the AI (Max) would have avoided reaching this state earlier. The algorithm stops exploring further moves down this specific branch (*alpha cutoff*).
2.  While exploring the AI's potential moves (Max node): If the evaluation of a potential AI move yields a score greater than or equal to `beta`, the AI knows this move looks promising, but it also knows the opponent (Min) already has a way to achieve a better (lower) outcome for themselves (`beta`) higher up the tree. Thus, the opponent would never let the game reach this state. The algorithm stops exploring further down this branch (*beta cutoff*).

By eliminating redundant calculations for board states that wouldn't be chosen under optimal play, Alpha-Beta Pruning makes deeper searches in Connect 4 computationally feasible, leading to much stronger AI performance.

### 4.2 Negamax

#### Concept
Negamax is a variant of the Minimax algorithm specifically tailored for zero-sum games (where one player's gain is exactly the other player's loss), like Connect 4. It simplifies the Minimax logic by observing that maximizing the score for the current player is equivalent to minimizing the score for the opponent. The core principle is often expressed as `max(score for Player A) = -max(score for Player B)`.

Instead of having separate logic for maximizing and minimizing players, Negamax always attempts to maximize the score from the perspective of the player whose turn it currently is.

#### How it Works in Connect 4

1.  **Single Recursive Function:** A single function explores the game tree.
2.  **Score Negation:** When the function recursively calls itself for the opponent's move, the score returned by that call (which represents the best outcome for the *opponent*) is negated (`-eval_score`). This flips the score back to the perspective of the *current* player.
3.  **Maximization:** The function always chooses the move that leads to the maximum score *after* negation.
4.  **Alpha-Beta Pruning:** Pruning works similarly to Minimax, but requires careful handling of the bounds. When making the recursive call, the alpha and beta bounds are swapped and negated (`negamax(..., -beta, -alpha, ...)`). This reflects the change in perspective for the opponent.
5.  **Heuristic & Terminal Evaluation:** Uses the same heuristic evaluation function (`score_position`) as Minimax. Terminal state scores (win/loss/draw) are calculated relative to the *original* AI player making the top-level call.

#### Pros
*   **Code Conciseness:** Often results in slightly less code than a traditional Minimax implementation due to the unified recursive logic (no explicit `if maximizing_player:` check).
*   **Theoretical Equivalence:** If implemented correctly with the same heuristic, depth, and pruning logic, it yields the exact same decisions as Minimax.

#### Cons
*   **Conceptual Nuance:** The score negation and alpha-beta bound swapping can be slightly less intuitive to understand initially compared to the explicit Max/Min player roles in Minimax.

#### Integration with CP
Negamax integrates with CP in the same way as Minimax:
*   Uses CP-verified win checks to identify terminal nodes.
*   Relies on CP-verified valid move generation to know which branches to explore.
*   The heuristic function (`score_position`), while likely procedural for speed, evaluates features (like N-in-a-row, threats) that can be formally defined using CP constraints.

### 4.3 Monte Carlo Tree Search (MCTS)

#### Concept
Monte Carlo Tree Search is a probabilistic search algorithm often used for game AI. Unlike Minimax/Negamax which attempt exhaustive search up to a fixed depth using heuristics, MCTS uses repeated random simulations (playouts) to estimate the value of different moves. It builds a search tree incrementally, focusing more time on promising areas of the game.

#### Key Steps (The MCTS Cycle)
MCTS runs for a fixed number of iterations or a time budget. Each iteration involves four phases:

1.  **Selection:** Starting from the root node (current board state), traverse down the existing tree by repeatedly selecting child nodes. The selection typically uses a strategy like UCB1 (Upper Confidence Bound 1 for Trees), which balances **exploitation** (choosing nodes that have historically led to good results) and **exploration** (choosing nodes that haven't been visited much).
2.  **Expansion:** Once a node is reached that is not fully expanded (i.e., has potential moves not yet added as children in the tree) and is not a terminal state, add one (or more) new child nodes to the tree corresponding to an unexplored move.
3.  **Simulation (Playout):** From the newly expanded node (or the selected leaf node if it was terminal), simulate a complete game by making random (or semi-random using a 'default policy') moves for both players until a terminal state (win/loss/draw) is reached.
4.  **Backpropagation:** Take the result of the simulation (+1 for win, -1 for loss, 0 for draw, usually relative to the player whose turn it was at the root) and update the statistics (visit count 'N' and value 'Q') of all nodes along the path taken during the Selection phase, from the simulated node back up to the root.

#### How it Works in Connect 4

*   **Tree Structure:** Each node holds a Connect 4 board state, visit count (N), total outcome value (Q), and links to parent/children.
*   **Simulations:** Random or slightly improved playouts determine game outcomes.
*   **Move Choice:** After the allocated iterations/time, the algorithm examines the direct children of the root node. The move corresponding to the child node that is considered 'best' (often the most visited, as it's statistically robust) is chosen as the AI's move.

#### Pros
*   **No Heuristic Required (Baseline):** Basic MCTS only needs win/loss/draw results, not a complex board evaluation function (though heuristics can enhance it).
*   **Asymmetric Growth:** Focuses search effort on more promising lines of play.
*   **Anytime Algorithm:** Can be stopped at any time (after any number of iterations) and provide the best move found so far.
*   **Effective in Complex Games:** Works well even with very large state spaces or branching factors where Minimax becomes infeasible.

#### Cons
*   **Computationally Intensive:** Requires many simulations (iterations) for strong play.
*   **Simulation Quality:** Performance depends on the quality and speed of the simulation phase. Purely random playouts can sometimes miss obvious tactical threats if not run for enough iterations (mitigated by improved default policies).
*   **Non-Deterministic (Usually):** Due to the randomness in simulations and potentially in tie-breaking during selection, it might not always choose the same move in the same situation.

#### Integration with CP
MCTS relies heavily on fast and accurate game rule enforcement, making CP integration valuable:
*   **Simulation Termination:** Needs rapid CP-verified win/loss/draw checks to end simulations.
*   **Valid Moves:** Requires CP-verified valid move generation during both the Expansion phase (creating new nodes) and the Simulation phase (choosing random/policy moves).

## 5. Expert System (Leo algo)

An Expert System approach, encodes human expert knowledge about Connect 4 strategy into a set of rules or heuristics. Unlike simpler rule-based agents, an expert system might incorporate more complex pattern recognition and strategic principles.

**Concept:** The core idea is to mimic how a skilled human player might reason about the game. This involves identifying critical board configurations, potential threats, opportunities to create winning lines, and general positional advantages.

**How it Might Work in Connect 4:**
*   **Knowledge Base:** Contains rules and patterns representing expert strategies (e.g., "If setting up a double threat is possible, prioritize it", "Avoid placing a piece below an opponent's potential winning spot if it enables their win", "Recognize and counter common trap patterns").
*   **Inference Engine:** Applies the rules from the knowledge base to the current board state to evaluate potential moves.
*   **Pattern Matching:** May involve sophisticated algorithms to detect complex arrangements of pieces that signal threats or opportunities (e.g., identifying potential 'forks' or 'ZUG patterns').
*   **Move Selection:** Chooses the move deemed best according to the evaluation performed by the inference engine based on the applied rules and recognized patterns.

**Pros:**
*   **Strong Performance (Potentially):** Can achieve high performance if the encoded knowledge is comprehensive and accurate.
*   **Explainability (Potentially):** The reasoning behind a move might be traceable back to specific rules, making it more understandable than purely learned models.

**Cons:**
*   **Knowledge Engineering Bottleneck:** Creating and refining the expert rules is time-consuming and requires deep domain expertise.
*   **Brittleness:** May perform poorly in situations not explicitly covered by the rules.
*   **Complexity:** Implementing sophisticated pattern matching and rule inference can be complex.

**Integration with CP:** CP can assist in formally defining the complex patterns and conditions that the expert system needs to recognize, making the rule implementation more robust.

## 6. Pattern-Based Agent

A Pattern-Based agent focuses specifically on recognizing predefined geometric or strategic patterns on the Connect 4 board to guide its move selection. It's closely related to Expert Systems and Rule-Based Heuristics but emphasizes the detection of specific configurations of discs.

**Concept:** The agent maintains a library of known patterns, each associated with a strategic value or action. It scans the board for occurrences of these patterns and chooses a move based on the presence and significance of detected patterns.

**How it Might Work in Connect 4:**
*   **Pattern Library:** Contains representations of patterns like:
    *   Immediate winning lines (3-in-a-row with an open space).
    *   Opponent's immediate threats.
    *   Potential forks (setting up two simultaneous winning threats).
    *   Common traps or sequences.
    *   Lines of 2 or 3 with open ends.
*   **Board Scanning:** Algorithms to efficiently search the current board for instances of patterns in the library.
*   **Evaluation/Action:** Each pattern might have a score or a recommended move (e.g., play in the winning spot, block the threat, play in the fork-creating spot). The agent aggregates this information, possibly prioritizing certain patterns over others, to select its final move.

**Pros:**
*   **Targeted Strategy:** Can be very effective at executing specific tactics and recognizing common threats.
*   **Efficiency:** Pattern matching can sometimes be faster than deep tree searches for specific tactical situations.

**Cons:**
*   **Incompleteness:** The agent's strength is limited by the comprehensiveness of its pattern library. It might miss novel situations or long-term strategic considerations not captured by patterns.
*   **Development Effort:** Defining and implementing the pattern detection logic requires careful design and potentially significant effort.

**Integration with CP:** CP can be used to formally define the patterns themselves as constraint satisfaction problems, aiding in their precise identification on the board.

## 7. Genetic Algorithm Agent

A Genetic Algorithm (GA) approach uses principles inspired by biological evolution to discover effective strategies or parameters for playing Connect 4. Instead of relying on predefined rules or direct game tree search during play, a GA typically optimizes components of a strategy offline.

**Concept:**
GAs operate on a population of 'individuals', where each individual represents a potential solution or strategy component. These individuals are evaluated based on their 'fitness' – typically how well they perform in playing Connect 4. Fitter individuals are preferentially selected to 'reproduce', creating a new generation of solutions by combining aspects of the parents (crossover) and introducing small random changes (mutation). Over many generations, this process tends to evolve the population towards higher-performing solutions.

**How it Might Work in Connect 4:**
*   **Representation (Chromosome):** An individual's 'chromosome' needs to encode a strategy. Common approaches include:
    *   **Heuristic Function Weights:** The chromosome could be a set of numerical weights assigned to different features of the board state (e.g., the value of having two pieces in a row, three pieces in a row, controlling the center column, blocking opponent threats). The agent then uses a function that calculates a score for potential moves based on these weighted features.
    *   **Rule-Based Parameters:** If the agent uses a set of rules, the chromosome could encode parameters that control how these rules are applied or prioritized.
    *   **(Advanced) Policy Representation:** It could encode parameters of a function (like a neural network) that directly maps board states to moves.
*   **Fitness Evaluation (Offline):** The core of the GA process. To determine how 'fit' a set of weights or parameters is, an agent using them must play many games. Opponents could be other individuals in the population, fixed benchmark agents, or previous versions of the evolving agent. Fitness is usually measured by win rate or average score achieved over these games.
*   **Evolutionary Operators:** Standard GA operators are applied iteratively:
    *   **Selection:** Choosing fitter individuals to become parents.
    *   **Crossover:** Creating offspring by combining genetic material (e.g., parts of the weight vectors) from two parents.
    *   **Mutation:** Introducing small, random changes to offspring's chromosomes (e.g., slightly altering a weight) to maintain diversity and explore new possibilities.

**Runtime Agent Behavior:**
Once the offline GA process has converged or run for a sufficient time, the best-evolved chromosome (e.g., the most effective set of heuristic weights) is extracted. The runtime agent then uses this *fixed* set of parameters to make decisions during a game. Typically, it evaluates all valid moves by applying the evolved heuristic function to the resulting board states and selects the move leading to the best score, possibly incorporating basic checks for immediate wins or losses.

**Pros:**
*   **Optimization Power:** Effective at finding good sets of parameters within a defined strategy space (like weights for a heuristic).
*   **Discovery:** Can potentially discover non-obvious interactions between strategic features or effective parameter balances.
*   **Adaptability (during training):** The population adapts over generations to the challenges posed by opponents used in fitness evaluation.

**Cons:**
*   **Computational Cost (Training):** The evolutionary process requires significant computation, involving potentially millions of game simulations for fitness evaluation.
*   **Representation Sensitivity:** The effectiveness heavily depends on choosing a good way to represent the strategy (the chromosome) and defining relevant features (if evolving heuristic weights).
*   **Parameter Tuning:** GAs have their own set of parameters (population size, crossover/mutation rates) that require tuning for optimal performance.
*   **Static Runtime Strategy:** Once trained, the agent uses fixed parameters; it doesn't adapt further during a single game beyond its heuristic evaluation.

**Integration with CP:** Constraint Programming is vital for the fitness evaluation phase within the GA, ensuring that the numerous simulated games adhere strictly to Connect 4 rules (valid moves, win/loss/draw detection).

## 8. Reinforcement Learning (RL)

### Concept
Reinforcement Learning is a machine learning paradigm where an agent learns to make decisions by taking actions in an environment to maximize a cumulative reward signal. The agent isn't told *what* action to take, but instead discovers which actions yield the most reward through trial and error.

### Key Components
*   **Agent:** The learner or decision-maker (our Connect 4 player).
*   **Environment:** The external system the agent interacts with (the Connect 4 game).
*   **State (s):** A representation of the environment's current situation (the board configuration).
*   **Action (a):** A choice the agent can make (dropping a disc in a column).
*   **Reward (r):** Feedback from the environment indicating the immediate consequence of an action (e.g., +1 for winning, -1 for losing, 0 for other moves).
*   **Policy (π):** The agent's strategy for choosing actions based on states.
*   **Value Function (V(s) or Q(s,a)):** Estimates the expected long-term return (cumulative reward) from a state or state-action pair.

### Relevance to Connect 4
RL allows an agent to learn to play Connect 4 potentially without prior knowledge of optimal strategies, simply by playing many games (often against itself or variations of itself) and learning from the outcomes.

### 8.1 Deep Q-Learning (DQN)

Deep Q-Learning is a popular RL algorithm that uses a deep neural network to approximate the optimal action-value function, known as Q*(s, a). This function represents the maximum expected future reward achievable from state 's' by taking action 'a' and following the optimal policy thereafter.

**How it Works (Briefly):**

1.  **Q-Network:** A neural network takes the current state 's' as input and outputs estimated Q-values for each possible action 'a' in that state.
2.  **Experience Replay:** The agent's experiences (state, action, reward, next_state) tuples are stored in a memory buffer. During training, mini-batches are randomly sampled from this buffer. This breaks temporal correlations and improves learning stability.
3.  **Target Network:** A separate neural network (a periodically updated copy of the main Q-network) is used to calculate the target Q-values for the learning update. This target is `r + γ * max_a'(Q_target(s', a'))`, where `γ` (gamma) is the discount factor for future rewards, `s'` is the next state, and `Q_target` is the value from the target network. Using a fixed target network for a period stabilizes training.
4.  **Learning:** The main Q-network is updated using gradient descent to minimize the difference (e.g., Mean Squared Error) between its predicted Q(s, a) and the calculated target Q-value.
5.  **Exploration:** An exploration strategy (like epsilon-greedy, where the agent takes a random action with probability epsilon, otherwise chooses the action with the highest Q-value) is used to balance exploring new actions and exploiting known good actions.

#### 8.1.1 DQN with a Simple Neural Network (MLP)

*   **Input:** The Connect 4 board (6x7 grid) is typically flattened into a 1D vector (e.g., 42 elements). Each element represents a cell, possibly encoded as 0 (empty), 1 (player 1), -1 (player 2).
*   **Network:** A Multi-Layer Perceptron (MLP) with one or more hidden dense layers (using activation functions like ReLU) followed by an output layer.
*   **Output:** The output layer has 7 neurons, one for each column. The value of each output neuron represents the estimated Q-value for dropping a disc in that column.
*   **Pros:** Relatively simple to implement.
*   **Cons:** Flattening the input loses the spatial relationships between cells (adjacency, lines), which are crucial in Connect 4. The network must learn these relationships implicitly, which can be inefficient.

#### 8.1.2 DQN with a Convolutional Neural Network (CNN)

*   **Input:** The board is treated as a 2D grid (e.g., 6x7). Often, multiple input *channels* are used to represent the state more effectively (e.g., one 6x7 channel indicating positions of player 1's pieces, another for player 2's pieces, maybe a third channel indicating whose turn it is).
*   **Network:** Starts with one or more convolutional layers. These layers use filters (kernels) to detect spatial patterns (like horizontal, vertical, diagonal lines, or local configurations) across the board. Pooling layers might be used to reduce dimensionality. The output from the convolutional/pooling layers is then typically flattened and fed into one or more dense layers (MLP style).
*   **Output:** Similar to the MLP approach, the final output layer has 7 neurons representing the Q-values for each column.
*   **Pros:** CNNs are designed to recognize spatial hierarchies and patterns, making them well-suited for board games like Connect 4. They can learn relevant features more efficiently than MLPs processing flattened input.
*   **Cons:** More complex architecture and more computationally intensive to train than a simple MLP.

In [ ]:
# Code cell for potential DQN structure (pseudo-code or library calls)
# E.g. (Conceptual):
# model = create_cnn_q_network(input_shape=(6, 7, num_channels), output_size=7)
# target_model = create_cnn_q_network(input_shape=(6, 7, num_channels), output_size=7)
# replay_buffer = ReplayBuffer(capacity=10000)
# optimizer = Adam(learning_rate=0.001)
# 
# for episode in range(num_episodes):
#     state = env.reset()
#     done = False
#     while not done:
#         action = select_action_epsilon_greedy(state, model, epsilon)
#         next_state, reward, done, _ = env.step(action)
#         replay_buffer.add(state, action, reward, next_state, done)
#         state = next_state
#         
#         if len(replay_buffer) > batch_size:
#             sample_batch = replay_buffer.sample(batch_size)
#             train_step(model, target_model, sample_batch, optimizer, gamma)
#             
#     # Update target network periodically
#     if episode % target_update_frequency == 0:
#         target_model.set_weights(model.get_weights())
#     
#     # Decay epsilon, log metrics, etc.

## 9. Performance Analysis and Benchmarking

This section presents the results of benchmarking various Connect 4 AI agents against each other. Each agent played a total of 140 games, split evenly between playing as Player 1 (starting player) and Player 2. In pairwise matchups, each agent played 10 games against every other agent as Player 1 and 10 games as Player 2.

### Overall Performance Ranking

The overall performance, ranked by win percentage across all 140 games played by each agent, is shown below:

```text
| Rank | AI Algorithm    | Win % | Wins | Losses | Draws | Total Games |
|------|-----------------|-------|------|--------|-------|-------------|
| 1    | Minimax         | 100.0 | 140  | 0      | 0     | 140         |
| 2    | Genetic         | 75.0  | 105  | 34     | 1     | 140         |
| 3    | Expert System   | 65.7  | 92   | 48     | 0     | 140         |
| 4    | MonteCarlo      | 60.0  | 84   | 55     | 1     | 140         |
| 5    | Pattern-Based   | 54.3  | 76   | 64     | 0     | 140         |
| 6    | Negamax         | 27.9  | 39   | 101    | 0     | 140         |
| 7    | Always First    | 11.4  | 16   | 124    | 0     | 140         |
| 8    | Random          | 5.0   | 7    | 133    | 0     | 140         |
```


**Observations:**
*   **Minimax Dominance:** The Minimax agent achieved a perfect 100% win rate, demonstrating its optimal play in this deterministic game against the tested opponents.
*   **Strong Performers:** The Genetic algorithm and the Expert System ('Leo algo') performed very well, securing 75% and 65.7% win rates respectively.
*   **Mid-Tier:** Monte Carlo and Pattern-Based agents show respectable performance around the 50-60% win rate mark.
*   **Weak Performers:** Negamax (likely due to implementation details or search depth compared to Minimax), Always First, and Random agents performed poorly, as expected for simpler or flawed strategies.

<center>
    <figure>
        <img src="benchmark_results\performance.png" width="800">
    </figure>
</center>


### Pairwise Matchup Performance

The heatmap below shows the win percentage of the Row player (playing as P1) against the Column player (playing as P2) across 10 games for each specific matchup.

<center>
    <figure>
        <img src="benchmark_results\heatmap.png" width="800">
    </figure>
</center>


**Observations:**
*   **Minimax Unbeatable:** The Minimax agent won 100% of its games, regardless of whether it played as P1 or P2 (the 0% entries in the Minimax *row* indicate the *opponent's* win rate when Minimax played as P2 was 0%).
*   **Strong vs. Weak:** Genetic, Leo algo, MonteCarlo, and Pattern-Based agents consistently achieve high win rates (often 100%) against the weaker Always First, Negamax, and Random agents.
*   **Competitive Matchups:** 
    *   Genetic vs MonteCarlo: Genetic wins 60% as P1.
    *   Leo algo vs MonteCarlo: Leo algo wins 70% as P1.
    *   MonteCarlo vs Leo algo: 50% win rate (likely P2 wins here).
    *   Pattern-Based vs Leo algo: Pattern-Based wins 100% as P1.
    *   Pattern-Based vs MonteCarlo: Pattern-Based wins 70% as P1.
*   **Negamax Struggles:** The Negamax agent loses most matchups, only showing strength against Random (90% win rate as P1).
*   **Always First & Random:** These baseline agents perform very poorly against all but each other (Always First beats Random 70% of the time as P1).


### Benchmarking Summary

The benchmark results clearly establish Minimax as the superior algorithm in this set, achieving perfect play. Genetic, Leo algo (Expert System), MonteCarlo, and Pattern-Based agents form a competitive middle tier, demonstrating significantly better-than-random strategies. Negamax, Always First, and Random lag considerably behind. The near-absence of errors suggests reliable agent implementations, with the single Monte Carlo timeout being a minor exception possibly related to its computational demands.

## 10. Bibliography

### Minimax
* https://en.wikipedia.org/wiki/Minimax
* https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning

### RL
* https://fr.wikipedia.org/wiki/Q-learning
* https://huggingface.co/learn/deep-rl-course/unit3/deep-q-algorithm
* https://www.youtube.com/watch?v=U9nkd2jt3b8
